In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_selection import RFE
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.svm import SVC

In [3]:
train_binary_fs = pd.read_csv('/content/drive/MyDrive/ML-Data/scaled_train_binary_feature_selection.csv')
test_binary_fs = pd.read_csv('/content/drive/MyDrive/ML-Data/scaled_test_binary_feature_selection.csv')

#train_binary = pd.read_csv('/content/drive/MyDrive/ML-Data/scaled_train_binary.csv')
#test_binary = pd.read_csv('/content/drive/MyDrive/ML-Data/scaled_test_binary.csv')

#train_fs = pd.read_csv('/content/drive/MyDrive/ML-Data/scaled_train_feature_selection.csv')
#test_fs = pd.read_csv('/content/drive/MyDrive/ML-Data/scaled_test_feature_selection.csv')

#train = pd.read_csv('/content/drive/MyDrive/ML-Data/scaled_train.csv')
#test = pd.read_csv('/content/drive/MyDrive/ML-Data/scaled_test.csv')

def get_XY(data):
  X = data.drop(['labels'] , axis=1).values
  y = data['labels'].values

  return X,y

#X_train_binary, y_train_binary = get_XY(train_binary)
#X_test_binary, y_test_binary = get_XY(test_binary)

In [4]:
#X_train, y_train = get_XY(train)
#X_test, y_test = get_XY(test)

#X_train_binary, y_train_binary = get_XY(train_binary)
#X_test_binary, y_test_binary = get_XY(test_binary)

#X_train_fs, y_train_fs = get_XY(train_fs)
#X_test_fs, y_test_fs = get_XY(test_fs)

X_train_binary_fs, y_train_binary_fs = get_XY(train_binary_fs)
X_test_binary_fs, y_test_binary_fs = get_XY(test_binary_fs)

In [5]:
def svm(X_trn, y_trn, X_tst, y_tst):
  rbf_svm = SVC(C=0.1, gamma='scale', kernel='rbf')

  rbf_svm.fit(X_trn, y_trn)

  y_pred = rbf_svm.predict(X_tst)

  print("Accuracy:",metrics.accuracy_score(y_tst, y_pred))
  return y_pred

In [6]:
def rfc(X_trn, y_trn, X_tst, y_tst):
  rf_classifier = RandomForestClassifier(n_estimators=200, max_depth=20, min_samples_leaf=2)

  rf_classifier.fit(X_trn, y_trn)

  y_pred = rf_classifier.predict(X_tst)

  accuracy = metrics.accuracy_score(y_tst, y_pred)
  print("Test Set Accuracy:", accuracy)

  return y_pred

In [7]:
#pred_binary_svm = svm(X_train_binary, y_train_binary, X_test_binary, y_test_binary)
pred_binary_svm_fs = svm(X_train_binary_fs, y_train_binary_fs, X_test_binary_fs, y_test_binary_fs)

#pred_binary_rfc = rfc(X_train_binary, y_train_binary, X_test_binary, y_test_binary)
pred_binary_rfc_fs = rfc(X_train_binary_fs, y_train_binary_fs, X_test_binary_fs, y_test_binary_fs)

#pred_svm = svm(X_train, y_train, X_test, y_test)
#pred_svm_fs = svm(X_train_fs, y_train_fs, X_test_fs, y_test_fs)

#pred_rfc = rfc(X_train, y_train, X_test, y_test)
#pred_rfc_fs = rfc(X_train_fs, y_train_fs, X_test_fs, y_test_fs)

Accuracy: 0.9056511710432931
Accuracy: 0.9688028387035317
Test Set Accuracy: 0.9058286018452804
Test Set Accuracy: 0.9999285561191684
Accuracy: 0.8940738112136267
Accuracy: 0.8873314407381121
Test Set Accuracy: 0.8773509581263307
Test Set Accuracy: 0.8754879347054648


In [27]:
predictions = {
    #"SVM (Binary)": pred_binary_svm,
    "SVM (Binary + Feature Selection)": pred_binary_svm_fs,
    #"RFC (Binary)": pred_binary_rfc,
    "RFC (Binary + Feature Selection)": pred_binary_rfc_fs,
    #"SVM (Multi Class)": pred_svm,
    #"SVM (Multi Class + Feature Selection)": pred_svm_fs,
    #"RFC (Multi Class)": pred_rfc,
    #"RFC (Multi Class + Feature Selection)": pred_rfc_fs
}

In [28]:
labels = {
    #"SVM (Binary)": y_test_binary,
    "SVM (Binary + Feature Selection)": y_test_binary_fs,
    #"RFC (Binary)": y_test_binary,
    "RFC (Binary + Feature Selection)": y_test_binary_fs,
    #"SVM (Multi Class)": y_test,
    #"SVM (Multi Class + Feature Selection)": y_test_fs,
    #"RFC (Multi Class)": y_test,
    #"RFC (Multi Class + Feature Selection)": y_test_fs
}

In [25]:
def plot_evaluation_metrics(y_true, y_pred):
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))

    # Confusion Matrix
    cm = metrics.confusion_matrix(y_true, y_pred)
    axes[0].imshow(cm, cmap=plt.cm.Blues, interpolation='nearest')
    axes[0].set(title='Confusion Matrix', xlabel='Predicted label', ylabel='True label')
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            axes[0].text(j, i, format(cm[i, j], 'd'), horizontalalignment="center", color="white" if cm[i, j] > cm.max() / 2 else "black")

    report = metrics.classification_report(y_true, y_pred)

    # ROC Curve
    fpr, tpr, thresholds = metrics.roc_curve(y_true, y_pred)
    roc_auc = metrics.auc(fpr, tpr)
    axes[1].plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
    axes[1].plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    axes[1].set(title='Receiver Operating Characteristic (ROC) Curve', xlabel='False Positive Rate', ylabel='True Positive Rate')
    axes[1].legend(loc="lower right")

    # Precision-Recall Curve
    precision, recall, _ = metrics.precision_recall_curve(y_true, y_pred)
    pr_auc = metrics.auc(recall, precision)
    axes[2].plot(recall, precision, color='blue', lw=2, label='Precision-Recall curve (area = %0.2f)' % pr_auc)
    axes[2].set(title='Precision-Recall Curve', xlabel='Recall', ylabel='Precision')
    axes[2].legend(loc="lower left")

    plt.tight_layout()
    plt.show()

    return report


In [31]:
for x,y in predictions.items():
  report = plot_evaluation_metrics(labels[x], y)



SVM (Binary)
              precision    recall  f1-score   support

           0       0.98      0.83      0.90     11299
           1       0.85      0.98      0.91     11245

    accuracy                           0.91     22544
   macro avg       0.92      0.91      0.91     22544
weighted avg       0.92      0.91      0.91     22544

SVM (Binary + Feature Selection)
              precision    recall  f1-score   support

           0       0.98      0.95      0.97     58630
           1       0.96      0.98      0.97     67343

    accuracy                           0.97    125973
   macro avg       0.97      0.97      0.97    125973
weighted avg       0.97      0.97      0.97    125973

RFC (Binary)
              precision    recall  f1-score   support

           0       0.94      0.87      0.90     11299
           1       0.88      0.94      0.91     11245

    accuracy                           0.91     22544
   macro avg       0.91      0.91      0.91     22544
weighted avg   